<a href="https://colab.research.google.com/github/alifarah94/DS-Unit-2-Applied-Modeling/blob/master/Week_7_Day_2_applied_modeling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - _**[A Kaggle Master Explains Gradient Boosting](http://blog.kaggle.com/2017/01/23/a-kaggle-master-explains-gradient-boosting/)**_
  - [_An Introduction to Statistical Learning_](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf) Chapter 8
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)
  - _**[Boosting](https://www.youtube.com/watch?v=GM3CDQfQ4sw) (2.5 minute video)**_

In [0]:
import pandas as pd
#import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [0]:
# uploads and cleans the data
sumo_matches = pd.read_csv('https://query.data.world/s/kp5eazhvwdbrnhhyow5lm4kfywhyvg') # fight result dataframe
sumo_info = pd.read_csv('https://query.data.world/s/6gckbhyl6klbem3vs25chgcaw65gfa') # sumo info dataframe
sumo_info = sumo_info.dropna()
sumo_info_2 = sumo_info.drop(['rank','birth_date','rikishi','prev','prev_w','prev_l'],axis=1)
sumo_1 = sumo_info_2.copy()
sumo_2 = sumo_info_2.copy()
sumo_1 = sumo_1.rename(columns={"id":"rikishi1_id","weight": "rikishi1_weight","height":"rikishi1_height","heya": "rikishi1_heya","shusshin":"rikishi1_shusshin"})
sumo_2 = sumo_2.rename(columns={"id":"rikishi2_id","weight": "rikishi2_weight","height":"rikishi2_height","heya": "rikishi2_heya","shusshin":"rikishi2_shusshin"})
sumo_matches_1 = sumo_matches.loc[(sumo_matches.index%2)==0]
sumo_matches_rik1 = pd.merge(sumo_matches_1,sumo_1,how='left',on=['basho','rikishi1_id'])
sumo_matches_rik1_rik2 = pd.merge(sumo_matches_rik1,sumo_2,how='left',on=['basho','rikishi2_id'])
df = sumo_matches_rik1_rik2



In [0]:
df = df.dropna()

In [0]:
target = 'rikishi1_win'
features = df.drop(columns=['rikishi1_win','rikishi2_win'])

In [0]:
train, validate = train_test_split(df, train_size=0.80, test_size=0.20, 
                              stratify=df['rikishi1_win'], random_state=42)

In [0]:
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

In [4]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [5]:
import eli5
from eli5.sklearn import PermutationImportance 

Using TensorFlow backend.


In [30]:
# using sumo weight to predict fights

model.fit(x_train[['rikishi1_weight','rikishi2_weight']],y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
x_validate=validate[features.columns]
y_validate = validate[target]
x_train = train[features.columns]
y_train = train[target]

In [27]:
x_validate[['rikishi1_weight','rikishi2_weight']]

,rikishi1_weight,rikishi2_weight
52114,160.0,168.5
21268,239.0,175.0
101043,121.0,172.0
46542,152.0,150.0
98547,185.2,153.0
...,...,...
61069,140.0,131.0
3437,135.0,179.0
62051,138.4,139.0
13534,152.5,135.0


In [33]:
permuter = PermutationImportance(
    model,
    scoring='accuracy',
    n_iter=2,
    random_state=42
)

permuter.fit(x_validate[['rikishi1_weight','rikishi2_weight']],y_validate)

PermutationImportance(cv='prefit',
                      estimator=RandomForestClassifier(bootstrap=True,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                                                       min_impurity_decrease=0.0,
                                                       min_impurity_split=None,
                                                       min_samples_leaf=1,
                                                       min_samples_split=2,
                                                       min_weight_fraction_leaf=0.0,
                                                       n_estimators=100,
                                                     

In [34]:
permuter.feature_importances_

array([0.02484515, 0.02174824])